In [98]:
import librosa
import numpy as np
from scipy import signal
from moviepy.editor import VideoFileClip, concatenate_videoclips
import wave

In [130]:
def find_offset(within_file, find_file, window):
    y_within, sr_within = librosa.load(within_file, sr=None)
    y_find, _ = librosa.load(find_file, sr=sr_within)

    # c = signal.correlate(y_within, y_find[:sr_within*window], mode='valid', method='fft')
    c = signal.correlate(y_within, y_find, mode='valid', method='fft')
    # peak = np.argsort(c)[::-1][:10]
    peak = np.argsort(c)[::-1][:10000]
    offsets = []
    for p in peak:
        # offset = round(p / sr_within, 2)
        offsets.append(p)


    return offsets

def remove_close_values(arr):
    min_value = min(arr)  # 配列内の最小値を取得

    # 近い値を削除
    filtered_arr = [min_value]
    for num in arr:
        if abs(num - min_value) > 1:  # 1以内の範囲外の値を残す
            filtered_arr.append(num)

    return filtered_arr

In [131]:
offset = find_offset('./test/kitasota_2.wav', './videos/kill_audio_test_trim_2.wav', 10)
print(offset)

[80296966, 69601011, 153124293, 33453594, 33453593, 159037221, 80296965, 69601010, 153124292, 80296951, 80296955, 80296967, 80296976, 80296959, 80296991, 33453619, 33453604, 80296977, 80296981, 33453583, 80296973, 69601000, 80296940, 69600996, 33453579, 159037220, 80296944, 80296941, 69601036, 80296962, 69601021, 33453608, 33453587, 80296980, 80296972, 153124282, 80296956, 80296970, 80296919, 69601004, 153124278, 33453568, 153124294, 69601026, 80296930, 69600985, 80296987, 69601018, 33453601, 33453572, 80297002, 33453609, 33453615, 80296934, 33453600, 80296960, 80296998, 153124308, 153124286, 80296958, 69600989, 69601012, 69601022, 69601025, 153124304, 80296945, 33453630, 153124303, 80297013, 33453605, 153124318, 80296961, 69601032, 69601047, 153124300, 69601017, 69600986, 33453626, 33453598, 69601007, 69601015, 33453586, 33453590, 33453597, 80296950, 69600964, 80296969, 80296988, 80296926, 80296915, 80296952, 69601058, 69601001, 80296971, 69601043, 80296948, 80296929, 153124268, 33453

In [20]:
def getNearestValue(list, num):
    """
    概要: リストからある値に最も近い値を返却する関数
    @param list: データ配列
    @param num: 対象値
    @return 対象値に最も近い値
    """

    # リスト要素と対象値の差分を計算し最小値のインデックスを取得
    idx = np.abs(np.asarray(list) - num).argmin()
    return list[idx]

In [132]:
offset_new = offset


for i in range(len(offset_new)):
    num = offset_new[0]
    offset_new = np.delete(offset_new, 0)
    near_value = getNearestValue(offset_new, num)
    if (np.abs(near_value - num)) < 44100*10:
        continue
    else:
        offset_new = np.append(offset_new, num)
# if np.abs((getNearestValue(offset_new)) - offset_new[]) < 10:

print(offset_new)
print(len(offset_new))

times = []
for ofs in offset_new:
    off = round(ofs / 44100, 2)
    times.append(off)
print(times)
print(len(times))





[140989366  80297090  94142759 164246157 171837734  54442350  33452695
 138087683 159037146 134309270  64130285 103694643 122706552 100507710
 160238828  96700001 153055571  49081062 118586217 101882640  69600132
  30899011 139444990]
23
[3197.04, 1820.8, 2134.76, 3724.4, 3896.55, 1234.52, 758.56, 3131.24, 3606.28, 3045.56, 1454.2, 2351.35, 2782.46, 2279.09, 3633.53, 2192.74, 3470.65, 1112.95, 2689.03, 2310.26, 1578.23, 700.66, 3162.02]
23


In [105]:
filename = "./test/kitasota_2.wav"
w = wave.open(filename, 'rb')
data = w.readframes(w.getnframes())
w.close()

In [123]:
count = 0
idx = 0
times_new = times
for t in offset_new:
    data_trim = data[t:t+44100*5]
    # data_trim = data[96699006:96699006+44100*5]    # 全滅
    # data_trim = data[29205906:29205906+44100*5]    # ポッド
    fs = w.getframerate()
    s = (np.frombuffer(data_trim, dtype="int16") / 32767.0)[0:fs]
    avg = np.average(np.abs(s))
    std = np.std(s)
    print(idx, avg, std)
    if avg < 0.4 or std < 0.4:
        # times_new = np.delete(times_new, count)
        # print('invalid')
        count += 1
    idx += 1

print(count)
print(len(times_new))
print(times_new)

0 0.500266129706987 0.5778305295127085
1 0.09864244950276627 0.12539849539807207
2 0.10072746576581011 0.13531477050797472
3 0.044074296446282205 0.06273656445795345
4 0.01305008627188172 0.0176310895175361
5 0.022154275286782297 0.029771086375360557
6 0.12735285839750698 0.15556522869845654
7 0.0729212469516957 0.10857804077465766
8 0.04740639312255355 0.0669659353359589
9 0.49760984500818567 0.57491831336635
10 0.4985191969382946 0.5757426835932883
11 0.49895126498529746 0.5763593871005804
12 0.17362318858632658 0.22511015461450173
13 0.4989657498588086 0.5759223558361483
14 0.5011967227965031 0.5780459222744028
15 0.04970149714395886 0.0835898436879542
16 0.500148562166446 0.5779668679675466
17 0.4989186932237214 0.5764105403746925
18 0.1238997935467816 0.17975978365052236
19 0.11256234858822829 0.14238420887247105
20 0.49930569491303506 0.576712177127939
21 0.09015139256789176 0.11194534177998078
22 0.49835165862562764 0.5760351671029
23 0.08422134860393736 0.1497127661390178
14
24

In [133]:
segments = []

for start_time in times:
    segments.append((start_time-3, start_time+3))

print(segments)

[(3194.04, 3200.04), (1817.8, 1823.8), (2131.76, 2137.76), (3721.4, 3727.4), (3893.55, 3899.55), (1231.52, 1237.52), (755.56, 761.56), (3128.24, 3134.24), (3603.28, 3609.28), (3042.56, 3048.56), (1451.2, 1457.2), (2348.35, 2354.35), (2779.46, 2785.46), (2276.09, 2282.09), (3630.53, 3636.53), (2189.74, 2195.74), (3467.65, 3473.65), (1109.95, 1115.95), (2686.03, 2692.03), (2307.26, 2313.26), (1575.23, 1581.23), (697.66, 703.66), (3159.02, 3165.02)]


In [134]:
from moviepy.editor import VideoFileClip, concatenate_videoclips

def cut_video(input_file, output_file, segments):
    clips = []
    # enumerate(tqdm(segments, desc='Processing', unit='segment')):
    for i, segment in enumerate(segments):
        start_time = segment[0]
        end_time = segment[1]
        clip = VideoFileClip(input_file).subclip(start_time, end_time)
        clips.append(clip)

    final_clip = concatenate_videoclips(clips)
    final_clip.write_videofile(output_file, audio_codec='aac')


# 切り取りたい区間を指定します（開始時間と終了時間のペアのリスト）


# 入力ファイルと出力ファイルのパスを指定します
input_file = './videos/kitasota_2.mp4'
output_file = 'output.mp4'

# 動画の切り取りと連結を実行します
cut_video(input_file, output_file, segments)


Moviepy - Building video output.mp4.
MoviePy - Writing audio in outputTEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video output.mp4



Moviepy - Done !
Moviepy - video ready output.mp4
